In [1]:
from pandas import Series, DataFrame
import pandas as pd
import re
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
#read the excel file and get the BOM mapping tab
xls = pd.ExcelFile('PostponementProject Data Package.xlsx')
df1 = pd.read_excel(xls, 'BOM.Mapping')
df1[:10]

,Part Description,Related Part Description,Mfg. Area
0,ORANGE-FG117,NaN,FG
1,ORANGE-FG121,NaN,FG
2,WHITE-FG127,NaN,FG
3,WHITE-FG123,NaN,FG
4,PURPLE-FG17,PURPLE-FG23,FG
5,PURPLE-FG17,PURPLE-FG21,FG
6,PURPLE-FG17,PURPLE-FG22,FG
7,PURPLE-FG12,NaN,FG
8,PURPLE-FG11,NaN,FG
9,PURPLE-FG13,NaN,FG


In [4]:
#select all the brown part among the table
df_brown = df1[df1['Part Description'].str.match('BROWN')]
df_brown[:10]

,Part Description,Related Part Description,Mfg. Area
132,BROWN-FG149,NaN,FG
133,BROWN-FG145,NaN,FG
134,BROWN-FG146,NaN,FG
135,BROWN-FG150,NaN,FG
136,BROWN-FG147,NaN,FG
137,BROWN-FG148,NaN,FG
372,BROWN-ASM712,BROWN-TST710,ASM
373,BROWN-ASM712,BROWN-TST711,ASM
374,BROWN-ASM702,BROWN-TST700,ASM
375,BROWN-ASM702,BROWN-TST701,ASM


In [5]:
#create a list of FG in Brown
Brown_FG = df_brown[df_brown['Mfg. Area'] == "FG"]['Part Description'].astype(str).tolist()
Brown_FG

['BROWN-FG149',
 'BROWN-FG145',
 'BROWN-FG146',
 'BROWN-FG150',
 'BROWN-FG147',
 'BROWN-FG148']

In [21]:
#build function to record steps for each FG
def build_step (step_real, tree):
    step_list = ['FG','MOD & FG','TST & FG','TST','ASM & FG','ASM','PRB','FAB']
    for step in step_list:
        if step_real == step:
            tree[step_list.index(step)] = 1
    return tree

In [22]:
# different cycle time for some Design ID, prob for brown is 21
cytime = [1,3,3,7,3,14,21,1]

In [23]:
#trace back from FG and record the mfg step and the part description
FG_cycletime = []
for prod in Brown_FG:
    prod_tree = [1,0,0,0,0,0,0,0]
    prod_ID = prod
    step_real = 'FG'
    while step_real != 'FAB':
        product_track = df_brown[df_brown['Related Part Description'] == prod_ID]
        step_real = product_track['Mfg. Area'].astype(str).iloc[0]
        print step_real
        prod_tree = build_step(step_real, prod_tree)
        prod_ID = product_track['Part Description'].astype(str).iloc[0]
        print prod_tree
        print prod_ID
    prod_cytime = sum(map(lambda i,j:i*j, prod_tree, cytime))
    FG_cycletime.append(prod_cytime)
print Brown_FG
print FG_cycletime

TST & FG
[1, 0, 1, 0, 0, 0, 0, 0]
BROWN-TST & FG713
TST
[1, 0, 1, 1, 0, 0, 0, 0]
BROWN-TST710
ASM
[1, 0, 1, 1, 0, 1, 0, 0]
BROWN-ASM712
PRB
[1, 0, 1, 1, 0, 1, 1, 0]
BROWN-PRB715
FAB
[1, 0, 1, 1, 0, 1, 1, 1]
BROWN
TST & FG
[1, 0, 1, 0, 0, 0, 0, 0]
BROWN-TST & FG704
TST
[1, 0, 1, 1, 0, 0, 0, 0]
BROWN-TST701
ASM
[1, 0, 1, 1, 0, 1, 0, 0]
BROWN-ASM702
PRB
[1, 0, 1, 1, 0, 1, 1, 0]
BROWN-PRB715
FAB
[1, 0, 1, 1, 0, 1, 1, 1]
BROWN
TST & FG
[1, 0, 1, 0, 0, 0, 0, 0]
BROWN-TST & FG703
TST
[1, 0, 1, 1, 0, 0, 0, 0]
BROWN-TST700
ASM
[1, 0, 1, 1, 0, 1, 0, 0]
BROWN-ASM702
PRB
[1, 0, 1, 1, 0, 1, 1, 0]
BROWN-PRB715
FAB
[1, 0, 1, 1, 0, 1, 1, 1]
BROWN
TST & FG
[1, 0, 1, 0, 0, 0, 0, 0]
BROWN-TST & FG714
TST
[1, 0, 1, 1, 0, 0, 0, 0]
BROWN-TST711
ASM
[1, 0, 1, 1, 0, 1, 0, 0]
BROWN-ASM712
PRB
[1, 0, 1, 1, 0, 1, 1, 0]
BROWN-PRB715
FAB
[1, 0, 1, 1, 0, 1, 1, 1]
BROWN
TST & FG
[1, 0, 1, 0, 0, 0, 0, 0]
BROWN-TST & FG708
TST
[1, 0, 1, 1, 0, 0, 0, 0]
BROWN-TST705
ASM
[1, 0, 1, 1, 0, 1, 0, 0]
BROWN-ASM707
PRB
[1, 0, 1

In [12]:
#build id map
def build_step_name (step_real,ID, tree):
    step_list = ['FG','MOD & FG','TST & FG','TST','ASM & FG','ASM','PRB','FAB']
    for step in step_list:
        if step_real == step:
            tree[step_list.index(step)] = ID
    return tree

In [16]:
# create id map for each FG
step_list_list = []
for prod in Brown_FG:
    prod_tree = [prod,0,0,0,0,0,0,0]
    prod_ID = prod
    step_real = 'FG'
    while step_real != 'FAB':
        product_track = df_brown[df_brown['Related Part Description'] == prod_ID]
        step_real = product_track['Mfg. Area'].astype(str).iloc[0]
        prod_ID = product_track['Part Description'].astype(str).iloc[0]
        prod_tree = build_step_name(step_real,prod_ID,prod_tree)
    #prod_cytime = sum(map(lambda i,j:i*j, prod_tree, cytime))
    #FG_cycletime.append(prod_cytime)
    step_list_list.append(prod_tree)
print step_list_list

[['BROWN-FG149', 0, 'BROWN-TST & FG713', 'BROWN-TST710', 0, 'BROWN-ASM712', 'BROWN-PRB715', 'BROWN'], ['BROWN-FG145', 0, 'BROWN-TST & FG704', 'BROWN-TST701', 0, 'BROWN-ASM702', 'BROWN-PRB715', 'BROWN'], ['BROWN-FG146', 0, 'BROWN-TST & FG703', 'BROWN-TST700', 0, 'BROWN-ASM702', 'BROWN-PRB715', 'BROWN'], ['BROWN-FG150', 0, 'BROWN-TST & FG714', 'BROWN-TST711', 0, 'BROWN-ASM712', 'BROWN-PRB715', 'BROWN'], ['BROWN-FG147', 0, 'BROWN-TST & FG708', 'BROWN-TST705', 0, 'BROWN-ASM707', 'BROWN-PRB715', 'BROWN'], ['BROWN-FG148', 0, 'BROWN-TST & FG709', 'BROWN-TST706', 0, 'BROWN-ASM707', 'BROWN-PRB715', 'BROWN']]


In [20]:
#Reorganize all the list to form a table
step_list = ['FG','MOD & FG','TST & FG','TST','ASM & FG','ASM','PRB','FAB']
Brown_table = pd.DataFrame.from_records(step_list_list,columns = step_list)
Brown_table['TOTAL LEADTIME'] = pd.Series(FG_cycletime).values
Brown_table

,FG,MOD & FG,TST & FG,TST,ASM & FG,ASM,PRB,FAB,TOTAL LEADTIME
0,BROWN-FG149,0,BROWN-TST & FG713,BROWN-TST710,0,BROWN-ASM712,BROWN-PRB715,BROWN,47
1,BROWN-FG145,0,BROWN-TST & FG704,BROWN-TST701,0,BROWN-ASM702,BROWN-PRB715,BROWN,47
2,BROWN-FG146,0,BROWN-TST & FG703,BROWN-TST700,0,BROWN-ASM702,BROWN-PRB715,BROWN,47
3,BROWN-FG150,0,BROWN-TST & FG714,BROWN-TST711,0,BROWN-ASM712,BROWN-PRB715,BROWN,47
4,BROWN-FG147,0,BROWN-TST & FG708,BROWN-TST705,0,BROWN-ASM707,BROWN-PRB715,BROWN,47
5,BROWN-FG148,0,BROWN-TST & FG709,BROWN-TST706,0,BROWN-ASM707,BROWN-PRB715,BROWN,47
